In [ ]:
'''
Aim- to segment features set from the aerial images

1. Read the images and divide it into smaller size with the computer can handel and easy process
2. saving the images without lable=0 (Back_ground) to reduce the computation time for the model
3. Create the training and test data set.


'''

In [1]:
# importing the necessary library
import os
import cv2
import numpy as np

from matplotlib import pyplot as plt
from patchify import patchify
from PIL import Image
import random

import tensorflow as tf
from tensorflow import keras
import segmentation_models as sm
from tensorflow.keras.metrics import MeanIoU


Segmentation Models: using `keras` framework.


In [9]:
#Quick understanding of the dataset for the one images
temp_img = cv2.imread(r"C:\Python310\tf\Scripts\Project_main\landcover.ai.v1\images\M-33-7-A-d-3-2.tif") #3 channels / spectral bands
#plt.imshow(temp_img[:,:,2]) #View each channel...

temp_mask = cv2.imread(r"C:\Python310\tf\Scripts\Project_main\landcover.ai.v1\masks\M-33-7-A-d-3-2.tif") #3 channels but all same. 
labels, count = np.unique(temp_mask[:,:,0], return_counts=True) #Check for each channel. All chanels are identical
print("Labels are: ", labels, " and the counts are: ", count)

Labels are:  [0 1 2 3 4]  and the counts are:  [31180689   324909 51703029  3286107  1795186]


In [11]:
root_directory = 'landcover.ai.v1/'

patch_size = 256

# Dividing the large image & respective mask into patch size 256x256x3

In [15]:
#divide all images into patches of 256x256x3. 
img_dir=root_directory+"images/"
for path, subdirs, files in os.walk(img_dir):
    #print(path)  
    dirname = path.split(os.path.sep)[-1]
    #print(dirname)
    images = os.listdir(path)  #List of all image names in this subdirectory
    #print(images)
    for i, image_name in enumerate(images):  
        if image_name.endswith(".tif"):
            #print(image_name)
            image = cv2.imread(path+"/"+image_name, 1)  #Read each image as BGR
            SIZE_X = (image.shape[1]//patch_size)*patch_size #Nearest size divisible by our patch size
            SIZE_Y = (image.shape[0]//patch_size)*patch_size #Nearest size divisible by our patch size
            image = Image.fromarray(image)
            image = image.crop((0 ,0, SIZE_X, SIZE_Y))  #Crop from top left corner
            #image = image.resize((SIZE_X, SIZE_Y))  #Try not to resize for semantic segmentation
            image = np.array(image)             
   
            #Extract patches from each image
            print("Now patchifying image:", path+"/"+image_name)
            patches_img = patchify(image, (256, 256, 3), step=256)  #Step=256 for 256 patches means no overlap
    
            for i in range(patches_img.shape[0]):
                for j in range(patches_img.shape[1]):
                    
                    single_patch_img = patches_img[i,j,:,:]
                    #single_patch_img = (single_patch_img.astype('float32')) / 255. #We will preprocess using one of the backbones
                    single_patch_img = single_patch_img[0] #Drop the extra unecessary dimension that patchify adds.                               
                    
                    cv2.imwrite(root_directory+"256_patches/images/"+
                               image_name+"patch_"+str(i)+str(j)+".tif", single_patch_img)
                    #image_dataset.append(single_patch_img)
            


Now patchifying image: landcover.ai.v1/images//M-33-20-D-c-4-2.tif
Now patchifying image: landcover.ai.v1/images//M-33-20-D-d-3-3.tif
Now patchifying image: landcover.ai.v1/images//M-33-32-B-b-4-4.tif
Now patchifying image: landcover.ai.v1/images//M-33-48-A-c-4-4.tif
Now patchifying image: landcover.ai.v1/images//M-33-7-A-d-2-3.tif
Now patchifying image: landcover.ai.v1/images//M-33-7-A-d-3-2.tif
Now patchifying image: landcover.ai.v1/images//M-34-32-B-a-4-3.tif
Now patchifying image: landcover.ai.v1/images//M-34-32-B-b-1-3.tif
Now patchifying image: landcover.ai.v1/images//M-34-5-D-d-4-2.tif
Now patchifying image: landcover.ai.v1/images//M-34-51-C-b-2-1.tif
Now patchifying image: landcover.ai.v1/images//M-34-51-C-d-4-1.tif
Now patchifying image: landcover.ai.v1/images//M-34-55-B-b-4-1.tif
Now patchifying image: landcover.ai.v1/images//M-34-56-A-b-1-4.tif
Now patchifying image: landcover.ai.v1/images//M-34-6-A-d-2-2.tif
Now patchifying image: landcover.ai.v1/images//M-34-65-D-a-4-4.tif

In [16]:
 #Now do the same as above for masks
 #For this specific dataset we could have added masks to the above code as masks have extension png
mask_dir=root_directory+"masks/"
for path, subdirs, files in os.walk(mask_dir):
    #print(path)  
    dirname = path.split(os.path.sep)[-1]

    masks = os.listdir(path)  #List of all image names in this subdirectory
    for i, mask_name in enumerate(masks):  
        if mask_name.endswith(".tif"):           
            mask = cv2.imread(path+"/"+mask_name, 0)  #Read each image as Grey (or color but remember to map each color to an integer)
            SIZE_X = (mask.shape[1]//patch_size)*patch_size #Nearest size divisible by our patch size
            SIZE_Y = (mask.shape[0]//patch_size)*patch_size #Nearest size divisible by our patch size
            mask = Image.fromarray(mask)
            mask = mask.crop((0 ,0, SIZE_X, SIZE_Y))  #Crop from top left corner
            #mask = mask.resize((SIZE_X, SIZE_Y))  #Try not to resize for semantic segmentation
            mask = np.array(mask)             
   
            #Extract patches from each image
            print("Now patchifying mask:", path+"/"+mask_name)
            patches_mask = patchify(mask, (256, 256), step=256)  #Step=256 for 256 patches means no overlap
    
            for i in range(patches_mask.shape[0]):
                for j in range(patches_mask.shape[1]):
                    
                    single_patch_mask = patches_mask[i,j,:,:]
                    #single_patch_img = (single_patch_img.astype('float32')) / 255. #No need to scale masks, but you can do it if you want
                    #single_patch_mask = single_patch_mask[0] #Drop the extra unecessary dimension that patchify adds.                               
                    cv2.imwrite(root_directory+"256_patches/masks/"+
                               mask_name+"patch_"+str(i)+str(j)+".tif", single_patch_mask)

Now patchifying mask: landcover.ai.v1/masks//M-33-20-D-c-4-2.tif
Now patchifying mask: landcover.ai.v1/masks//M-33-20-D-d-3-3.tif
Now patchifying mask: landcover.ai.v1/masks//M-33-32-B-b-4-4.tif
Now patchifying mask: landcover.ai.v1/masks//M-33-48-A-c-4-4.tif
Now patchifying mask: landcover.ai.v1/masks//M-33-7-A-d-2-3.tif
Now patchifying mask: landcover.ai.v1/masks//M-33-7-A-d-3-2.tif
Now patchifying mask: landcover.ai.v1/masks//M-34-32-B-a-4-3.tif
Now patchifying mask: landcover.ai.v1/masks//M-34-32-B-b-1-3.tif
Now patchifying mask: landcover.ai.v1/masks//M-34-5-D-d-4-2.tif
Now patchifying mask: landcover.ai.v1/masks//M-34-51-C-b-2-1.tif
Now patchifying mask: landcover.ai.v1/masks//M-34-51-C-d-4-1.tif
Now patchifying mask: landcover.ai.v1/masks//M-34-55-B-b-4-1.tif
Now patchifying mask: landcover.ai.v1/masks//M-34-56-A-b-1-4.tif
Now patchifying mask: landcover.ai.v1/masks//M-34-6-A-d-2-2.tif
Now patchifying mask: landcover.ai.v1/masks//M-34-65-D-a-4-4.tif
Now patchifying mask: landcov

In [14]:
train_img_dir = "landcover.ai.v1/256_patches/images/"
train_mask_dir = "landcover.ai.v1/256_patches/masks/"

# Taking usefull images from the folder

In [18]:
img_list = os.listdir(train_img_dir)
msk_list = os.listdir(train_mask_dir)
#Now, let us copy images and masks with real information to a new folder.
# real information = if mask has decent amount of labels other than 0. 

useless=0  #Useless image counter
for img in range(len(img_list)):   #Using t1_list as all lists are of same size
    img_name=img_list[img]
    mask_name = msk_list[img]
    print("Now preparing image and masks number: ", img)
      
    temp_image=cv2.imread(train_img_dir+img_list[img], 1)
   
    temp_mask=cv2.imread(train_mask_dir+msk_list[img], 0)
    #temp_mask=temp_mask.astype(np.uint8)
    
    val, counts = np.unique(temp_mask, return_counts=True)
    
    if (1 - (counts[0]/counts.sum())) > 0.05:  #At least 5% useful area with labels that are not 0
        print("Save Me")
        cv2.imwrite('landcover.ai.v1/256_patches/images_with_useful_info/images/'+img_name, temp_image)
        cv2.imwrite('landcover.ai.v1/256_patches/images_with_useful_info/masks/'+mask_name, temp_mask)
        
    else:
        print("I am useless")   
        useless +=1

print("Total useful images are: ", len(img_list)-useless)  #21924
print("Total useless images are: ", useless) #19722

Now preparing image and masks number:  0
I am useless
Now preparing image and masks number:  1
Save Me
Now preparing image and masks number:  2
I am useless
Now preparing image and masks number:  3
Save Me
Now preparing image and masks number:  4
Save Me
Now preparing image and masks number:  5
Save Me
Now preparing image and masks number:  6
I am useless
Now preparing image and masks number:  7
I am useless
Now preparing image and masks number:  8
I am useless
Now preparing image and masks number:  9
I am useless
Now preparing image and masks number:  10
I am useless
Now preparing image and masks number:  11
I am useless
Now preparing image and masks number:  12
Save Me
Now preparing image and masks number:  13
I am useless
Now preparing image and masks number:  14
I am useless
Now preparing image and masks number:  15
Save Me
Now preparing image and masks number:  16
Save Me
Now preparing image and masks number:  17
Save Me
Now preparing image and masks number:  18
Save Me
Now prepar

Save Me
Now preparing image and masks number:  161
Save Me
Now preparing image and masks number:  162
Save Me
Now preparing image and masks number:  163
Save Me
Now preparing image and masks number:  164
I am useless
Now preparing image and masks number:  165
I am useless
Now preparing image and masks number:  166
Save Me
Now preparing image and masks number:  167
I am useless
Now preparing image and masks number:  168
Save Me
Now preparing image and masks number:  169
I am useless
Now preparing image and masks number:  170
I am useless
Now preparing image and masks number:  171
I am useless
Now preparing image and masks number:  172
I am useless
Now preparing image and masks number:  173
Save Me
Now preparing image and masks number:  174
Save Me
Now preparing image and masks number:  175
I am useless
Now preparing image and masks number:  176
I am useless
Now preparing image and masks number:  177
I am useless
Now preparing image and masks number:  178
I am useless
Now preparing image

I am useless
Now preparing image and masks number:  327
I am useless
Now preparing image and masks number:  328
Save Me
Now preparing image and masks number:  329
I am useless
Now preparing image and masks number:  330
I am useless
Now preparing image and masks number:  331
Save Me
Now preparing image and masks number:  332
Save Me
Now preparing image and masks number:  333
Save Me
Now preparing image and masks number:  334
I am useless
Now preparing image and masks number:  335
I am useless
Now preparing image and masks number:  336
Save Me
Now preparing image and masks number:  337
I am useless
Now preparing image and masks number:  338
Save Me
Now preparing image and masks number:  339
I am useless
Now preparing image and masks number:  340
I am useless
Now preparing image and masks number:  341
I am useless
Now preparing image and masks number:  342
I am useless
Now preparing image and masks number:  343
I am useless
Now preparing image and masks number:  344
I am useless
Now prepa

I am useless
Now preparing image and masks number:  480
Save Me
Now preparing image and masks number:  481
Save Me
Now preparing image and masks number:  482
Save Me
Now preparing image and masks number:  483
Save Me
Now preparing image and masks number:  484
Save Me
Now preparing image and masks number:  485
Save Me
Now preparing image and masks number:  486
I am useless
Now preparing image and masks number:  487
I am useless
Now preparing image and masks number:  488
I am useless
Now preparing image and masks number:  489
Save Me
Now preparing image and masks number:  490
Save Me
Now preparing image and masks number:  491
I am useless
Now preparing image and masks number:  492
Save Me
Now preparing image and masks number:  493
Save Me
Now preparing image and masks number:  494
Save Me
Now preparing image and masks number:  495
Save Me
Now preparing image and masks number:  496
I am useless
Now preparing image and masks number:  497
Save Me
Now preparing image and masks number:  498
S

Save Me
Now preparing image and masks number:  647
I am useless
Now preparing image and masks number:  648
Save Me
Now preparing image and masks number:  649
Save Me
Now preparing image and masks number:  650
Save Me
Now preparing image and masks number:  651
I am useless
Now preparing image and masks number:  652
I am useless
Now preparing image and masks number:  653
I am useless
Now preparing image and masks number:  654
Save Me
Now preparing image and masks number:  655
Save Me
Now preparing image and masks number:  656
Save Me
Now preparing image and masks number:  657
I am useless
Now preparing image and masks number:  658
I am useless
Now preparing image and masks number:  659
I am useless
Now preparing image and masks number:  660
Save Me
Now preparing image and masks number:  661
Save Me
Now preparing image and masks number:  662
Save Me
Now preparing image and masks number:  663
I am useless
Now preparing image and masks number:  664
I am useless
Now preparing image and masks

...

I am useless
Now preparing image and masks number:  40882
I am useless
Now preparing image and masks number:  40883
I am useless
Now preparing image and masks number:  40884
Save Me
Now preparing image and masks number:  40885
Save Me
Now preparing image and masks number:  40886
Save Me
Now preparing image and masks number:  40887
I am useless
Now preparing image and masks number:  40888
I am useless
Now preparing image and masks number:  40889
I am useless
Now preparing image and masks number:  40890
I am useless
Now preparing image and masks number:  40891
I am useless
Now preparing image and masks number:  40892
I am useless
Now preparing image and masks number:  40893
I am useless
Now preparing image and masks number:  40894
I am useless
Now preparing image and masks number:  40895
I am useless
Now preparing image and masks number:  40896
I am useless
Now preparing image and masks number:  40897
Save Me
Now preparing image and masks number:  40898
Save Me
Now preparing image and ma

Save Me
Now preparing image and masks number:  41037
Save Me
Now preparing image and masks number:  41038
Save Me
Now preparing image and masks number:  41039
I am useless
Now preparing image and masks number:  41040
I am useless
Now preparing image and masks number:  41041
I am useless
Now preparing image and masks number:  41042
I am useless
Now preparing image and masks number:  41043
I am useless
Now preparing image and masks number:  41044
I am useless
Now preparing image and masks number:  41045
I am useless
Now preparing image and masks number:  41046
I am useless
Now preparing image and masks number:  41047
I am useless
Now preparing image and masks number:  41048
I am useless
Now preparing image and masks number:  41049
Save Me
Now preparing image and masks number:  41050
I am useless
Now preparing image and masks number:  41051
I am useless
Now preparing image and masks number:  41052
I am useless
Now preparing image and masks number:  41053
Save Me
Now preparing image and ma

I am useless
Now preparing image and masks number:  41206
I am useless
Now preparing image and masks number:  41207
I am useless
Now preparing image and masks number:  41208
Save Me
Now preparing image and masks number:  41209
Save Me
Now preparing image and masks number:  41210
Save Me
Now preparing image and masks number:  41211
Save Me
Now preparing image and masks number:  41212
Save Me
Now preparing image and masks number:  41213
Save Me
Now preparing image and masks number:  41214
Save Me
Now preparing image and masks number:  41215
Save Me
Now preparing image and masks number:  41216
Save Me
Now preparing image and masks number:  41217
Save Me
Now preparing image and masks number:  41218
Save Me
Now preparing image and masks number:  41219
Save Me
Now preparing image and masks number:  41220
I am useless
Now preparing image and masks number:  41221
I am useless
Now preparing image and masks number:  41222
I am useless
Now preparing image and masks number:  41223
Save Me
Now prep

Now preparing image and masks number:  41368
Save Me
Now preparing image and masks number:  41369
Save Me
Now preparing image and masks number:  41370
I am useless
Now preparing image and masks number:  41371
I am useless
Now preparing image and masks number:  41372
Save Me
Now preparing image and masks number:  41373
Save Me
Now preparing image and masks number:  41374
Save Me
Now preparing image and masks number:  41375
Save Me
Now preparing image and masks number:  41376
Save Me
Now preparing image and masks number:  41377
Save Me
Now preparing image and masks number:  41378
Save Me
Now preparing image and masks number:  41379
Save Me
Now preparing image and masks number:  41380
I am useless
Now preparing image and masks number:  41381
Save Me
Now preparing image and masks number:  41382
Save Me
Now preparing image and masks number:  41383
Save Me
Now preparing image and masks number:  41384
Save Me
Now preparing image and masks number:  41385
Save Me
Now preparing image and masks n

Save Me
Now preparing image and masks number:  41525
Save Me
Now preparing image and masks number:  41526
I am useless
Now preparing image and masks number:  41527
I am useless
Now preparing image and masks number:  41528
I am useless
Now preparing image and masks number:  41529
Save Me
Now preparing image and masks number:  41530
Save Me
Now preparing image and masks number:  41531
Save Me
Now preparing image and masks number:  41532
Save Me
Now preparing image and masks number:  41533
Save Me
Now preparing image and masks number:  41534
Save Me
Now preparing image and masks number:  41535
Save Me
Now preparing image and masks number:  41536
Save Me
Now preparing image and masks number:  41537
Save Me
Now preparing image and masks number:  41538
Save Me
Now preparing image and masks number:  41539
Save Me
Now preparing image and masks number:  41540
I am useless
Now preparing image and masks number:  41541
Save Me
Now preparing image and masks number:  41542
Save Me
Now preparing imag

# Code for splitting folder into train, test, and val.

In [4]:
'''Once the new folders are created rename them and arrange in the format below to be used
for semantic segmentation using data generators. 

pip install split-folders'''

import splitfolders  # or import split_folders

input_folder = 'landcover.ai.v1/256_patches/images_with_useful_info/'
output_folder = 'landcover.ai.v1/data_for_training_and_testing/'
# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(.75, .25), group_prefix=None)

In [ ]:
#Now manually move folders around to bring them to the following structure.
"""
Your current directory structure:
Data/
    train/
        images/
            img1, img2, ...
        masks/
            msk1, msk2, ....
    val/
        images/
            img1, img2, ...
        masks/
            msk1, msk2, ....
        
Copy the folders around to the following structure... 


Data/
    train_images/
                train/
                    img1, img2, img3, ......
    
    train_masks/
                train/
                    msk1, msk, msk3, ......
                    
    val_images/
                val/
                    img1, img2, img3, ......                

    val_masks/
                val/
                    msk1, msk, msk3, ......
      
                    
"""